## 1) 전리뷰 크롤링

In [6]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time
# from bs4 import BeautifulSoup
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import pymysql
import sqlite3
import csv

# 버전 충돌
import collections
collections.Callable = collections.abc.Callable


In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000183731&dispCatNo=100000100010014&trackingCd=Cat100000100010014_MID&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%97%90%EC%84%BC%EC%8A%A4/%EC%84%B8%EB%9F%BC/%EC%95%B0%ED%94%8C_%EC%A0%84%EC%B2%B4__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1'
driver.get(url)

click_reviews= driver.find_element(By.CLASS_NAME, "goods_reputation")
click_reviews.click()
time.sleep(2)

id=[] # 작성ID
star_rate=[] # 별점
review=[] # 리뷰 내용

## 1 페이지
page_html = driver.page_source
soup = BeautifulSoup(page_html,'html.parser')
    
# 리뷰 리스트
reviews_lst = soup.select('div.review_list_wrap > ul.inner_list > li')

for idx in range(len(reviews_lst)):
    # 작성자
    id.append(reviews_lst[idx].select('a.id')[0].text)
    # 별점
    star_rate.append(reviews_lst[idx].select('span.review_point')[0].text)
    # 내용
    review.append(reviews_lst[idx].select('div.txt_inner')[0].text)

##################################### 2페이지부터 나머지~~#########################################################
page_num=1
count=1

while True:
    try:
        link = driver.find_element(By.XPATH,f'//*[@id="gdasContentsArea"]/div/div[7]/a[{page_num}]')
        link.click()
        time.sleep(3)
        
        page_html = driver.page_source
        soup = BeautifulSoup(page_html,'html.parser')
        
        # 리뷰 리스트
        reviews_lst = soup.select('div.review_list_wrap > ul.inner_list > li')
        print(len(reviews_lst))
        
        for idx in range(len(reviews_lst)):
            # 작성자
            id.append(reviews_lst[idx].select('a.id')[0].text)
            # 별점
            star_rate.append(reviews_lst[idx].select('span.review_point')[0].text)
            # 내용
            review.append(reviews_lst[idx].select('div.txt_inner')[0].text)
        
        # 페이지
        if count>=10:
            page_num = 2
            count=1
        else:
            count+=1
            page_num += 1
            
        
        
    except NoSuchElementException:
        print(f'{page_num}페이지를 찾을수 없습니다. 크롤링 끝-')
        break



10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11페이지를 찾을수 없습니다. 크롤링 끝-


In [3]:
print(len(id),len(star_rate),len(review))

1000 1000 1000


In [5]:
review_df = pd.DataFrame({'id':id,'star_rate':star_rate, 'review':review })

In [6]:
review_df.head()

,id,star_rate,review
0,간호산데여,5점만점에 5점,환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...
1,올영메이트인데요,5점만점에 5점,온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...
2,hareen,5점만점에 5점,아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!
3,콩이파더22,5점만점에 5점,리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...
4,이단콩,5점만점에 5점,리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...


In [33]:
# 별점 숫자로 변환
review_df['star_rate'] = review_df['star_rate'].apply(lambda x: int(x.split(' ')[-1].replace('점','')))

AttributeError: 'int' object has no attribute 'split'

In [2]:
review_df = pd.read_csv('./files/oliveyoung_raw_review.csv',encoding='utf8')
review_df

,brand,prod_name,id,star_rate,review
0,브이티,VT 리들샷 100 에센스,간호산데여,5,환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...
1,브이티,VT 리들샷 100 에센스,올영메이트인데요,5,온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...
2,브이티,VT 리들샷 100 에센스,hareen,5,아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!
3,브이티,VT 리들샷 100 에센스,콩이파더22,5,리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...
4,브이티,VT 리들샷 100 에센스,이단콩,5,리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...
...,...,...,...,...,...
995,브이티,VT 리들샷 100 에센스,도제,3,려드름 피부도 쓸수있다 했는디저는 딱히 효과가 없는듯해요ㅠㅍ
996,브이티,VT 리들샷 100 에센스,당토,2,자극은 그렇다치고... 이거 사용하니까 조그마한 좁쌀모양 피지가 얼굴 전체에 생겼습...
997,브이티,VT 리들샷 100 에센스,어도러블여왕님,4,계속 쓰다보니 점점 안 따가워지는 것 같아요 다음엔 더 센 걸로 사보려구요!
998,브이티,VT 리들샷 100 에센스,써보고좋으면짖음,5,아무래도 자극이 느껴지지만 자연스러운 현상이라고 하니 꾸준히 써볼 생각입니다


In [5]:
review_df['brand'] = '브이티'
review_df['prod_name'] = 'VT 리들샷 100 에센스'
review_df = review_df[['brand','prod_name','id','star_rate','review']]
review_df.head()

,brand,prod_name,id,star_rate,review
0,브이티,VT 리들샷 100 에센스,간호산데여,5,환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...
1,브이티,VT 리들샷 100 에센스,올영메이트인데요,5,온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...
2,브이티,VT 리들샷 100 에센스,hareen,5,아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!
3,브이티,VT 리들샷 100 에센스,콩이파더22,5,리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...
4,브이티,VT 리들샷 100 에센스,이단콩,5,리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...


In [12]:
import pandas as pd
import re


# 이모티콘 제거 함수 정의
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # 이모지
                           u"\U0001F300-\U0001F5FF"  # 이모티콘 & 기호
                           u"\U0001F680-\U0001F6FF"  # 트랜스포터 & 심볼
                           u"\U0001F700-\U0001F77F"  # 임베드 & 심볼 보충
                           u"\U0001F780-\U0001F7FF"  # 야구 보충
                           u"\U0001F800-\U0001F8FF"  # 아케이드 보충
                           u"\U0001F900-\U0001F9FF"  # 장난감 보충
                           u"\U0001FA00-\U0001FA6F"  # 음식 보충
                           u"\U0001FA70-\U0001FAFF"  # 공항 보충
                           u"\U0001FB00-\U0001FBFF"  # 추가 일본 보충
                           u"\U0001FC00-\U0001FCFF"  # 추가 한국 보충
                           u"\U0001F004-\U0001F0CF"  # 추가 스포츠 보충
                           u"\U0001F170-\U0001F251" 
                           u"\U0001F004-\U0001F0CF" 
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F004-\U0001F0CF" 
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F700-\U0001F77F" 
                           u"\U0001F780-\U0001F7FF" 
                           u"\U0001F800-\U0001F8FF" 
                           u"\U0001F900-\U0001F9FF" 
                           u"\U0001FA00-\U0001FA6F" 
                           u"\U0001FA70-\U0001FAFF" 
                           u"\U0001FB00-\U0001FBFF" 
                           u"\U0001FC00-\U0001FCFF" 
                           u"\U0001F004-\U0001F0CF" 
                           u"\U0001F170-\U0001F251" 
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# 'review' 열에서 이모티콘 제거
review_df['review'] = review_df['review'].apply(remove_emoji)

# 결과 데이터프레임 확인
print(review_df)


    brand       prod_name        id  star_rate  \
0     브이티  VT 리들샷 100 에센스     간호산데여          5   
1     브이티  VT 리들샷 100 에센스  올영메이트인데요          5   
2     브이티  VT 리들샷 100 에센스    hareen          5   
3     브이티  VT 리들샷 100 에센스    콩이파더22          5   
4     브이티  VT 리들샷 100 에센스       이단콩          5   
..    ...             ...       ...        ...   
995   브이티  VT 리들샷 100 에센스        도제          3   
996   브이티  VT 리들샷 100 에센스        당토          2   
997   브이티  VT 리들샷 100 에센스   어도러블여왕님          4   
998   브이티  VT 리들샷 100 에센스  써보고좋으면짖음          5   
999   브이티  VT 리들샷 100 에센스       아이온          5   

                                                review  
0    환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...  
1    온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...  
2    아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!  
3    리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...  
4    리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...  
..                                                 ...  


In [13]:
review_df['review']

0      환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...
1      온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...
2      아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!
3      리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...
4      리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...
                             ...                        
995                    려드름 피부도 쓸수있다 했는디저는 딱히 효과가 없는듯해요ㅠㅍ
996    자극은 그렇다치고... 이거 사용하니까 조그마한 좁쌀모양 피지가 얼굴 전체에 생겼습...
997           계속 쓰다보니 점점 안 따가워지는 것 같아요 다음엔 더 센 걸로 사보려구요!
998           아무래도 자극이 느껴지지만 자연스러운 현상이라고 하니 꾸준히 써볼 생각입니다
999                100은 그렇게 생각보다 따끔거리지않네요 다음엔 700을 사야겠어요
Name: review, Length: 1000, dtype: object

In [15]:
review_df.to_csv('./files/oliveyoung_raw_review.csv', index=False, encoding='utf-8-sig')

### DB에 밀어넣기

In [4]:
import pandas as pd
import pymysql
csv_file ='./output/oliveyoung_raw_review.csv'
df = pd.read_csv(csv_file)
df

,brand,prod_name,id,star_rate,review
0,브이티,VT 리들샷 100 에센스,간호산데여,5,환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...
1,브이티,VT 리들샷 100 에센스,올영메이트인데요,5,온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...
2,브이티,VT 리들샷 100 에센스,hareen,5,아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!
3,브이티,VT 리들샷 100 에센스,콩이파더22,5,리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...
4,브이티,VT 리들샷 100 에센스,이단콩,5,리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...
...,...,...,...,...,...
995,브이티,VT 리들샷 100 에센스,도제,3,려드름 피부도 쓸수있다 했는디저는 딱히 효과가 없는듯해요ㅠㅍ
996,브이티,VT 리들샷 100 에센스,당토,2,자극은 그렇다치고... 이거 사용하니까 조그마한 좁쌀모양 피지가 얼굴 전체에 생겼습...
997,브이티,VT 리들샷 100 에센스,어도러블여왕님,4,계속 쓰다보니 점점 안 따가워지는 것 같아요 다음엔 더 센 걸로 사보려구요!
998,브이티,VT 리들샷 100 에센스,써보고좋으면짖음,5,아무래도 자극이 느껴지지만 자연스러운 현상이라고 하니 꾸준히 써볼 생각입니다


In [7]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

f=open('./output/oliveyoung_raw_review.csv','r')

csvReader = list(csv.reader(f))
cur.execute("create table if not exists review_db ( brand VARCHAR(20), prod_name VARCHAR(40), id VARCHAR(20),\
star_rate INT, review TEXT, PRIMARY KEY (id))")


for data in csvReader[1:]:
    row1= data[0]
    row2= data[1]
    row3= data[2]
    row4= int(data[3])
    row5= data[4]
    
    print(row3,row4,type(row4))
    
    try:
        sql = """insert into review_db (brand, prod_name, id, star_rate, review) values(%s, %s, %s, %s, %s);"""
        cur.execute(sql,(row1,row2,row3,row4,row5))
    except:
        continue
    
f.close()
conn.commit()


conn.close()

간호산데여 5 <class 'int'>
올영메이트인데요 5 <class 'int'>
hareen 5 <class 'int'>
콩이파더22 5 <class 'int'>
이단콩 5 <class 'int'>
올올덕 5 <class 'int'>
안녕하숙까 5 <class 'int'>
jwwcat 5 <class 'int'>
비니수 5 <class 'int'>
Mizthecat 5 <class 'int'>
syjal 5 <class 'int'>
bear베어 5 <class 'int'>
냐얀야나냥 5 <class 'int'>
야야아야양 5 <class 'int'>
푸링짱 5 <class 'int'>
jio11 5 <class 'int'>
jio11 5 <class 'int'>
오롤롤 5 <class 'int'>
hq1 5 <class 'int'>
체리쿵 5 <class 'int'>
부타로니 5 <class 'int'>
코투닝 5 <class 'int'>
미미미mme 5 <class 'int'>
코코아녕 3 <class 'int'>
Likey 5 <class 'int'>
마리안네 4 <class 'int'>
경의선귀여운죄안된다 5 <class 'int'>
자신있써어어 5 <class 'int'>
푸링짱 5 <class 'int'>
김먕먕 5 <class 'int'>
메리크 5 <class 'int'>
O3OY 5 <class 'int'>
Channri 5 <class 'int'>
볼빵빵춘식이 5 <class 'int'>
bear베어 5 <class 'int'>
whvely 5 <class 'int'>
sol솔 3 <class 'int'>
충동구매니니즈 4 <class 'int'>
정연니 1 <class 'int'>
살랑이 4 <class 'int'>
Myzu 5 <class 'int'>
콩이맘맘 5 <class 'int'>
올영에진심인편 5 <class 'int'>
sol솔 3 <class 'int'>
올챙임 4 <class 'int'>
하늘을나는고래 5 <class 'i

In [ ]:
# # MySQL 연결 설정
# conn = sqlite3.connect('beaulol')

# # CSV 파일 경로
# csv_file_path = './files/oliveyoung_raw_review.csv'

# # 테이블 이름
# table_name = 'review_db'

# try:
#     # CSV 파일을 데이터프레임으로 읽기
#     df = pd.read_csv(csv_file_path)

#     # MySQL 데이터베이스로 데이터프레임 저장
#     df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

#     print(f"CSV 파일이 '{table_name}' 테이블로 MySQL 데이터베이스에 저장되었습니다.")
# except Exception as e:
#     print(f"오류 발생: {str(e)}")
# finally:
#     # 연결 닫기
#     conn.close()

CSV 파일이 'review_db' 테이블로 MySQL 데이터베이스에 저장되었습니다.


In [ ]:
# conn = sqlite3.connect('C:\ProgramData\MySQL\MySQL Server 8.0\Data\beaulol')
# cur = conn.cursor()
# cur.execute("""
#             CREATE TABLE IF NOT EXISTS prod_review (
#                 brand VARCHAR(20),
#                 prod_name VARCHAR(40), 
#                 id varchar(20),
#                 star_rate int,
#                 review text,
#                 PRIMARY KEY (id))
#             """)

# df.to_sql('prod_review',conn,if_exists='replace')

# conn.commit()
# conn.close()

In [ ]:
# reviews_lst = soup.select('div.review_list_wrap > ul.inner_list > li')
# # len(reviews_lst)

# # 작성자
# reviews_lst[0].select('a.id')[0].text
# # 별점
# reviews_lst[0].select('span.review_point')[0].text
# # 내용
# reviews_lst[0].select('div.txt_inner')[0].text

'환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 물론 자극이 있긴한데 효과가 너무 좋아서 걍 참고 사용합니당저는 울긋불긋 올라왔을 때 바르면 바로 진정되는 것 같더라구요? 이거 쓰고 요철도 진짜 많이 줄어들었어요!! 완전 겁나 지대 추천합니당특히 저는 생리 주기 다가오면 피부가 다 뒤집어지는 사람인데 니들샷 바르고 난 뒤부터는 안뒤집어졌어요:)그런 고민 있으신 분들껜 완전 추천드립니당~'